In [1]:
!pip install sklearn

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd


import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

from tensorflow.python.keras import Input, Model
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import load_model

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-1739ba8b1e2c>", line 7, in <module>
    import tensorflow as tf
  File "C:\Anaconda\lib\site-packages\tensorflow\__init__.py", line 101, in <module>
    from tensorflow_core import *
  File "C:\Anaconda\lib\site-packages\tensorflow_core\__init__.py", line 40, in <module>
    from tensorflow.python.tools import module_util as _module_util
ModuleNotFoundError: No module named 'tensorflow.python.tools'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ModuleNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception oc

ModuleNotFoundError: No module named 'tensorflow.python.tools'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cd

In [ ]:
#train = pd.read_csv('/content/drive/My Drive/star2_data/star_train.csv')
test= pd.read_csv('/content/drive/My Drive/star2/star_test.csv')
train= pd.read_csv('/content/drive/My Drive/star2/star_train.csv')

In [ ]:
train, val = train_test_split(train, test_size=0.2)
print(len(train), '훈련 샘플')
print(len(val), '검증 샘플')
print(len(test), '테스트 샘플')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
test['winner'] = 0

In [ ]:
test.head()

In [ ]:
# 판다스 데이터프레임으로부터 tf.data 데이터셋을 만들기 위한 함수
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('winner')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [ ]:
#feature_columns = []

# 수치형 열
#for header in train.columns:
#  feature_columns.append(feature_column.numeric_column(header))

In [ ]:
feature_columns = []

# 수치형 열
#여기 왜인지는 모르겠는데 train.columns하면 안돌아가더라구,,,그냥 2130개 중에서 몇개 뽑아서 일단 했는데,,,좀 보완해야할 것 같아!
for header in ['game_id',	'time',	'player0_species',	'player1_species',	'event_count_0',	'event_count_1', 'Camera_count_0',	'Camera_count_1',	'Selection_count_0',	'Selection_count_1',	'Ability_count_0',	'Ability_count_1',	'Right_Click_count_0',	'Right_Click_count_1',	'SetControlGroup_count_0',	'SetControlGroup_count_1',	'GetControlGroup_count_0',	'GetControlGroup_count_1',	'AddToControlGroup_count_0',	'AddToControlGroup_count_1',	'ControlGroup_count_0',	'ControlGroup_count_1',	'Camera_diff',	'Selection_diff',	'Ability_diff',	'Right_Click_diff',	'SetControlGroup_diff',	'GetControlGroup_diff',	'AddToControlGroup_diff',	'ControlGroup_diff',	'underTime9',	'Location_count_0',	'Target__None__count_0',	'Target__Medivac__count_0',	'Target__DestructibleSignsConstruction__count_0',	'Target__Assimilator__count_0',	'Target__Pylon__count_0',	'Target__SiegeTankSieged__count_0',	'Target__Nexus__count_0',	'Target__WarpPrism__count_0', '_ThorNormalMode_diff',	'_CancelUpgradeToMothership_diff',	'_EMPRound_diff',	'_UnburrowRoach_diff',	'_BurrowInfestor_diff',	'_TacticalNukeStrike_diff',	'_HealMedivac_diff',	'_UnburrowBaneling_diff',	'_HallucinateColossus_diff',	'_42A3__diff',	'_TrainInterceptor_diff',	'_HoldFireGhost_diff',	'_StimpackRedirect_diff',	'_StopGenerateCreep_diff',	'_EvolveAdrenalGlands_diff',	'_HallucinateVoidRay_diff',	'_4AFE__diff',	'_LoadTargetBunker_diff',	'_UpgradeGroundWeapons3_diff',	'_EnableVolatileBurst_diff',	'_BurrowRavagerDown_diff',	'_BurrowRavagerUp_diff',	'_ResearchTerranVehicleAndShipArmorsLevel2_diff',	'_PsionicStorm_diff',	'_EvolveNeuralParasite_diff',	'_HallucinateArchon_diff',	'player0_x',	'player0_y',	'player1_x',	'player1_y',	'player0_starting',	'player1_starting',	'map_0.0',	'map_1.0',	'map_2.0',	'map_3.0',	'map_4.0',	'map_5.0',	'map_6.0']:
  feature_columns.append(feature_column.numeric_column(header))

In [ ]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [ ]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [ ]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)

test data에서 'winner'열 삭제해야합니당~!!

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("정확도", accuracy)

In [ ]:
train_ds

In [ ]:
val_ds

In [ ]:
test_ds